# Preparation

In [4]:
import numpy as np
import pandas as pd

import numba

import iqplot

import bebi103.viz

import colorcet

import bokeh.io
from bokeh.io import export_svgs
from bokeh.layouts import column
from bokeh.layouts import gridplot
from bokeh.models import Span, Band, BoxAnnotation, Ray, Segment, Legend, Line, LegendItem, Label, Title
from bokeh.models.tools import HoverTool
hover = HoverTool()
import bokeh_catplot
bokeh.io.output_notebook()

import os, sys, subprocess

Loading BokehJS ...

In [2]:
ColorG = '#33a02c'
ColorH = '#e41a1c'

ColorABC = '#fdb462'
ColorABR = '#7570b3'
ColorAB = '#8dd3c7'
ColorA = '#bebada'
ColorBC = '#b3de69'
ColorB = '#b15928'
ColorC = '#fb8072'
ColorJK = '#80b1d3'
ColorJ = '#fccde5'
ColorK = '#ffd92f'
ColorT = '#bc80bd'
ColorTJ = '#1b9e77'
ColorTJK = '#d12d7f'
ColorTJR = '#377eb8'

ColorDEF = '#bebada'
ColorDES = '#F1C10F'
ColorDE = '#b15928'
ColorD = '#fdb462'
ColorEF = '#fb8072'
ColorE = '#8dd3c7'
ColorF = '#b3de69'
ColorMN = '#ffd92f'
ColorM = '#1b9e77'
ColorN = '#80b1d3'
ColorU = '#d12d7f'
ColorUM = '#fccde5'
ColorUMN = '#bc80bd'
ColorUMS = '#E7E200'

C:\Users\tzboz\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
varea_alpha=.15
line_plot_line_width=2

size_font_tick = '16pt'
size_font_label = '16pt'
size_font_title = '16pt'
size_font_legend = '16pt'
size_font_xtitle = '16pt'

In [5]:
df4a = pd.read_csv(os.path.join('../data/', "2nd NOS - Figure 4a.csv"))
df4b = pd.read_csv(os.path.join('../data/', "2nd NOS - Figure 4b.csv"))
df4c = pd.read_csv(os.path.join('../data/', "2nd NOS - Figure 4c.csv"))
df4d = pd.read_csv(os.path.join('../data/', "2nd NOS - Figure 4d.csv"))
df4e = pd.read_csv(os.path.join('../data/', "2nd NOS - Figure 4e.csv"))
df4f = pd.read_csv(os.path.join('../data/', "2nd POS - Figure 4f.csv"))
df4g = pd.read_csv(os.path.join('../data/', "2nd POS - Figure 4g.csv"))
df4h = pd.read_csv(os.path.join('../data/', "2nd POS - Figure 4h.csv"))
df4i = pd.read_csv(os.path.join('../data/', "2nd POS - Figure 4i.csv"))
df4j = pd.read_csv(os.path.join('../data/', "2nd POS - Figure 4j.csv"))

# Conduct bootstrapping for 95% confidence intervals

In [6]:
@numba.njit
def draw_bs_sample(data):
    """Draw a bootstrap sample from a 1D data set."""
    return np.random.choice(data, size=len(data))

In [7]:
def draw_bs_reps(data, stat_fun, size=1):
    """Draw boostrap replicates computed with stat_fun from 1D data set."""
    return np.array([stat_fun(draw_bs_sample(data)) for _ in range(size)])


@numba.njit
def draw_bs_reps_mean(data, size=1):
    """Draw boostrap replicates of the mean from 1D data set."""
    out = np.empty(size)
    for i in range(size):
        out[i] = np.mean(draw_bs_sample(data))
    return out


@numba.njit
def draw_bs_reps_median(data, size=1):
    """Draw boostrap replicates of the median from 1D data set."""
    out = np.empty(size)
    for i in range(size):
        out[i] = np.median(draw_bs_sample(data))
    return out


@numba.njit
def draw_bs_reps_std(data, size=1):
    """Draw boostrap replicates of the standard deviation from 1D data set."""
    out = np.empty(size)
    for i in range(size):
        out[i] = np.std(draw_bs_sample(data))
    return out

In [8]:
rg = np.random.default_rng()

# Set up Numpy arrays for convenience (also much better performance)
df4aH_values = df4a.loc[df4a['Stimulus'] == 'H-', 'US Exp'].values
df4aBH_values = df4a.loc[df4a['Stimulus'] == 'BH', 'US Exp'].values
df4aBC_values = df4a.loc[df4a['Stimulus'] == 'BC+', 'US Exp'].values

df4bH_values = df4b.loc[df4b['Stimulus'] == 'H-', 'US Exp'].values
df4bJH_values = df4b.loc[df4b['Stimulus'] == 'JH', 'US Exp'].values
df4bJK_values = df4b.loc[df4b['Stimulus'] == 'JK+', 'US Exp'].values

df4cH_values = df4c.loc[df4c['Stimulus'] == 'H-', 'US Exp'].values
df4cAH_values = df4c.loc[df4c['Stimulus'] == 'AH', 'US Exp'].values
df4cAG_values = df4c.loc[df4c['Stimulus'] == 'AG', 'US Exp'].values
df4cG_values = df4c.loc[df4c['Stimulus'] == 'G+', 'US Exp'].values

df4dG_values = df4d.loc[df4d['Stimulus'] == 'G+', 'US Exp'].values
df4dABG_values = df4d.loc[df4d['Stimulus'] == 'ABG', 'US Exp'].values
df4dJK_values = df4d.loc[df4d['Stimulus'] == 'JK+', 'US Exp'].values
df4dAJK_values = df4d.loc[df4d['Stimulus'] == 'AJK', 'US Exp'].values
df4dABC_values = df4d.loc[df4d['Stimulus'] == 'ABC-', 'US Exp'].values

df4eJK2_values = df4e.loc[df4e['Stimulus'] == 'JK+2', 'US Exp'].values
df4eAJK1_values = df4e.loc[df4e['Stimulus'] == 'AJK1', 'US Exp'].values
df4eAJK2_values = df4e.loc[df4e['Stimulus'] == 'AJK2', 'US Exp'].values
df4eTJK2_values = df4e.loc[df4e['Stimulus'] == 'TJK-2', 'US Exp'].values
df4eABC2_values = df4e.loc[df4e['Stimulus'] == 'ABC-2', 'US Exp'].values



df4fEF_values = df4f.loc[df4f['Stimulus'] == 'EF-', 'US Exp'].values
df4fEG_values = df4f.loc[df4f['Stimulus'] == 'EG', 'US Exp'].values
df4fG_values = df4f.loc[df4f['Stimulus'] == 'G+', 'US Exp'].values

df4gMN_values = df4g.loc[df4g['Stimulus'] == 'MN-', 'US Exp'].values
df4gMG_values = df4g.loc[df4g['Stimulus'] == 'MG', 'US Exp'].values
df4gG_values = df4g.loc[df4g['Stimulus'] == 'G+', 'US Exp'].values

df4hH_values = df4h.loc[df4h['Stimulus'] == 'H-', 'US Exp'].values
df4hDH_values = df4h.loc[df4h['Stimulus'] == 'DH', 'US Exp'].values
df4hDG_values = df4h.loc[df4h['Stimulus'] == 'DG', 'US Exp'].values
df4hG_values = df4h.loc[df4h['Stimulus'] == 'G+', 'US Exp'].values

df4iH_values = df4i.loc[df4i['Stimulus'] == 'H-', 'US Exp'].values
df4iDEH_values = df4i.loc[df4i['Stimulus'] == 'DEH', 'US Exp'].values
df4iMN_values = df4i.loc[df4i['Stimulus'] == 'MN-', 'US Exp'].values
df4iDMN_values = df4i.loc[df4i['Stimulus'] == 'DMN', 'US Exp'].values
df4iDEF_values = df4i.loc[df4i['Stimulus'] == 'DEF+', 'US Exp'].values

df4jMN2_values = df4j.loc[df4j['Stimulus'] == 'MN-2', 'US Exp'].values
df4jDMN1_values = df4j.loc[df4j['Stimulus'] == 'DMN1', 'US Exp'].values
df4jDMN2_values = df4j.loc[df4j['Stimulus'] == 'DMN2', 'US Exp'].values
df4jUMN2_values = df4j.loc[df4j['Stimulus'] == 'UMN+2', 'US Exp'].values
df4jDEF2_values = df4j.loc[df4j['Stimulus'] == 'DEF+2', 'US Exp'].values

In [9]:
bs_reps_mean_df4aH = draw_bs_reps_mean(df4aH_values, size=10000)
bs_reps_mean_df4aBH = draw_bs_reps_mean(df4aBH_values, size=10000)
bs_reps_mean_df4aBC = draw_bs_reps_mean(df4aBC_values, size=10000)

bs_reps_mean_df4bH = draw_bs_reps_mean(df4bH_values, size=10000)
bs_reps_mean_df4bJH = draw_bs_reps_mean(df4bJH_values, size=10000)
bs_reps_mean_df4bJK = draw_bs_reps_mean(df4bJK_values, size=10000)

bs_reps_mean_df4cH = draw_bs_reps_mean(df4cH_values, size=10000)
bs_reps_mean_df4cAH = draw_bs_reps_mean(df4cAH_values, size=10000)
bs_reps_mean_df4cAG = draw_bs_reps_mean(df4cAG_values, size=10000)
bs_reps_mean_df4cG = draw_bs_reps_mean(df4cG_values, size=10000)

bs_reps_mean_df4dG = draw_bs_reps_mean(df4dG_values, size=10000)
bs_reps_mean_df4dABG = draw_bs_reps_mean(df4dABG_values, size=10000)
bs_reps_mean_df4dJK = draw_bs_reps_mean(df4dJK_values, size=10000)
bs_reps_mean_df4dAJK = draw_bs_reps_mean(df4dAJK_values, size=10000)
bs_reps_mean_df4dABC = draw_bs_reps_mean(df4dABC_values, size=10000)

bs_reps_mean_df4eJK2 = draw_bs_reps_mean(df4eJK2_values, size=10000)
bs_reps_mean_df4eAJK1 = draw_bs_reps_mean(df4eAJK1_values, size=10000)
bs_reps_mean_df4eAJK2 = draw_bs_reps_mean(df4eAJK2_values, size=10000)
bs_reps_mean_df4eTJK2 = draw_bs_reps_mean(df4eTJK2_values, size=10000)
bs_reps_mean_df4eABC2 = draw_bs_reps_mean(df4eABC2_values, size=10000)



bs_reps_mean_df4fEF = draw_bs_reps_mean(df4fEF_values, size=10000)
bs_reps_mean_df4fEG = draw_bs_reps_mean(df4fEG_values, size=10000)
bs_reps_mean_df4fG = draw_bs_reps_mean(df4fG_values, size=10000)

bs_reps_mean_df4gMN = draw_bs_reps_mean(df4gMN_values, size=10000)
bs_reps_mean_df4gMG = draw_bs_reps_mean(df4gMG_values, size=10000)
bs_reps_mean_df4gG = draw_bs_reps_mean(df4gG_values, size=10000)

bs_reps_mean_df4hH = draw_bs_reps_mean(df4hH_values, size=10000)
bs_reps_mean_df4hDH = draw_bs_reps_mean(df4hDH_values, size=10000)
bs_reps_mean_df4hDG = draw_bs_reps_mean(df4hDG_values, size=10000)
bs_reps_mean_df4hG = draw_bs_reps_mean(df4hG_values, size=10000)

bs_reps_mean_df4iH = draw_bs_reps_mean(df4iH_values, size=10000)
bs_reps_mean_df4iDEH = draw_bs_reps_mean(df4iDEH_values, size=10000)
bs_reps_mean_df4iMN = draw_bs_reps_mean(df4iMN_values, size=10000)
bs_reps_mean_df4iDMN = draw_bs_reps_mean(df4iDMN_values, size=10000)
bs_reps_mean_df4iDEF = draw_bs_reps_mean(df4iDEF_values, size=10000)

bs_reps_mean_df4jMN2 = draw_bs_reps_mean(df4jMN2_values, size=10000)
bs_reps_mean_df4jDMN1 = draw_bs_reps_mean(df4jDMN1_values, size=10000)
bs_reps_mean_df4jDMN2 = draw_bs_reps_mean(df4jDMN2_values, size=10000)
bs_reps_mean_df4jUMN2 = draw_bs_reps_mean(df4jUMN2_values, size=10000)
bs_reps_mean_df4jDEF2 = draw_bs_reps_mean(df4jDEF2_values, size=10000)

In [10]:
# 95% confidence intervals
mean_df4aH_conf_int = np.percentile(bs_reps_mean_df4aH, [2.5, 97.5])
mean_df4aBH_conf_int = np.percentile(bs_reps_mean_df4aBH, [2.5, 97.5])
mean_df4aBC_conf_int = np.percentile(bs_reps_mean_df4aBC, [2.5, 97.5])

mean_df4bH_conf_int = np.percentile(bs_reps_mean_df4bH, [2.5, 97.5])
mean_df4bJH_conf_int = np.percentile(bs_reps_mean_df4bJH, [2.5, 97.5])
mean_df4bJK_conf_int = np.percentile(bs_reps_mean_df4bJK, [2.5, 97.5])

mean_df4cH_conf_int = np.percentile(bs_reps_mean_df4cH, [2.5, 97.5])
mean_df4cAH_conf_int = np.percentile(bs_reps_mean_df4cAH, [2.5, 97.5])
mean_df4cAG_conf_int = np.percentile(bs_reps_mean_df4cAG, [2.5, 97.5])
mean_df4cG_conf_int = np.percentile(bs_reps_mean_df4cG, [2.5, 97.5])

mean_df4dG_conf_int = np.percentile(bs_reps_mean_df4dG, [2.5, 97.5])
mean_df4dABG_conf_int = np.percentile(bs_reps_mean_df4dABG, [2.5, 97.5])
mean_df4dJK_conf_int = np.percentile(bs_reps_mean_df4dJK, [2.5, 97.5])
mean_df4dAJK_conf_int = np.percentile(bs_reps_mean_df4dAJK, [2.5, 97.5])
mean_df4dABC_conf_int = np.percentile(bs_reps_mean_df4dABC, [2.5, 97.5])

mean_df4eJK2_conf_int = np.percentile(bs_reps_mean_df4eJK2, [2.5, 97.5])
mean_df4eAJK1_conf_int = np.percentile(bs_reps_mean_df4eAJK1, [2.5, 97.5])
mean_df4eAJK2_conf_int = np.percentile(bs_reps_mean_df4eAJK2, [2.5, 97.5])
mean_df4eTJK2_conf_int = np.percentile(bs_reps_mean_df4eTJK2, [2.5, 97.5])
mean_df4eABC2_conf_int = np.percentile(bs_reps_mean_df4eABC2, [2.5, 97.5])



mean_df4fEF_conf_int = np.percentile(bs_reps_mean_df4fEF, [2.5, 97.5])
mean_df4fEG_conf_int = np.percentile(bs_reps_mean_df4fEG, [2.5, 97.5])
mean_df4fG_conf_int = np.percentile(bs_reps_mean_df4fG, [2.5, 97.5])

mean_df4gMN_conf_int = np.percentile(bs_reps_mean_df4gMN, [2.5, 97.5])
mean_df4gMG_conf_int = np.percentile(bs_reps_mean_df4gMG, [2.5, 97.5])
mean_df4gG_conf_int = np.percentile(bs_reps_mean_df4gG, [2.5, 97.5])

mean_df4hH_conf_int = np.percentile(bs_reps_mean_df4hH, [2.5, 97.5])
mean_df4hDH_conf_int = np.percentile(bs_reps_mean_df4hDH, [2.5, 97.5])
mean_df4hDG_conf_int = np.percentile(bs_reps_mean_df4hDG, [2.5, 97.5])
mean_df4hG_conf_int = np.percentile(bs_reps_mean_df4hG, [2.5, 97.5])

mean_df4iH_conf_int = np.percentile(bs_reps_mean_df4iH, [2.5, 97.5])
mean_df4iDEH_conf_int = np.percentile(bs_reps_mean_df4iDEH, [2.5, 97.5])
mean_df4iMN_conf_int = np.percentile(bs_reps_mean_df4iMN, [2.5, 97.5])
mean_df4iDMN_conf_int = np.percentile(bs_reps_mean_df4iDMN, [2.5, 97.5])
mean_df4iDEF_conf_int = np.percentile(bs_reps_mean_df4iDEF, [2.5, 97.5])

mean_df4jMN2_conf_int = np.percentile(bs_reps_mean_df4jMN2, [2.5, 97.5])
mean_df4jDMN1_conf_int = np.percentile(bs_reps_mean_df4jDMN1, [2.5, 97.5])
mean_df4jDMN2_conf_int = np.percentile(bs_reps_mean_df4jDMN2, [2.5, 97.5])
mean_df4jUMN2_conf_int = np.percentile(bs_reps_mean_df4jUMN2, [2.5, 97.5])
mean_df4jDEF2_conf_int = np.percentile(bs_reps_mean_df4jDEF2, [2.5, 97.5])

C:\Users\tzboz\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Figures

In [11]:
df4a_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
    y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4a_ecdf.add_layout(Title(
    text='POS1 to CS-', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4a_ecdf.add_layout(Title(
    text='Lack of Transfer of Unambiguous', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)


df4a_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4a_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4a,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorBC, ColorBC, ColorH],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4a_ecdf
)

#BC+
df4a_ecdf.line(
    x=[0,1],
    y=[np.mean(df4aBC_values),np.mean(df4aBC_values)],
    line_color=ColorBC,
    line_width=line_plot_line_width,
    legend='BC+',
)

df4a_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4aBC_conf_int[1],
    y2=mean_df4aBC_conf_int[0],
    color=ColorBC,
    legend_label='BC+',
    alpha=varea_alpha,
)

#BH
df4a_ecdf.line(
    x=[0,1],
    y=[np.mean(df4aBH_values),np.mean(df4aBH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='BH',
    line_dash='dotted',
)

df4a_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4aBH_conf_int[1],
    y2=mean_df4aBH_conf_int[0],
    color=ColorH,
    legend_label='BH',
    alpha=varea_alpha,
)

#H-
df4a_ecdf.line(
    x=[0,1],
    y=[np.mean(df4aH_values),np.mean(df4aH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='H-',
)

df4a_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4aH_conf_int[1],
    y2=mean_df4aH_conf_int[0],
    color=ColorH,
    legend_label='H-',
    alpha=varea_alpha,
)

df4a_ecdf.title.text_font_size = size_font_title
df4a_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4a_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4a_ecdf.legend.label_text_font_size = size_font_legend
df4a_ecdf.axis.major_label_text_font_size = size_font_tick

# Toolbar_location = None
# bokeh.io.show(df4a_ecdf)


In [12]:
df4b_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     title='Lack of Transfer of Ambiguous POS1 to CS-',
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
    y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4b_ecdf.add_layout(Title(
    text='POS1 to CS-', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4b_ecdf.add_layout(Title(
    text='Lack of Transfer of Ambiguous', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)

df4b_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4b_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4b,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorJK, ColorJK, ColorH],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4b_ecdf
)

#H-
df4b_ecdf.line(
    x=[0,1],
    y=[np.mean(df4bH_values),np.mean(df4bH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='H-',
)

df4b_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4bH_conf_int[1],
    y2=mean_df4bH_conf_int[0],
    color=ColorH,
    legend_label='H-',
    alpha=varea_alpha,
)

#JH
df4b_ecdf.line(
    x=[0,1],
    y=[np.mean(df4bJH_values),np.mean(df4bJH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='JH',
    line_dash='dotted',
)

df4b_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4bJH_conf_int[1],
    y2=mean_df4bJH_conf_int[0],
    color=ColorH,
    legend_label='JH',
    alpha=varea_alpha,
)

#JK
df4b_ecdf.line(
    x=[0,1],
    y=[np.mean(df4bJK_values),np.mean(df4bJK_values)],
    line_color=ColorJK,
    line_width=line_plot_line_width,
    legend='JK+',
)

df4b_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4bJK_conf_int[1],
    y2=mean_df4bJK_conf_int[0],
    color=ColorJK,
    legend_label='JK+',
    alpha=varea_alpha,
)

df4b_ecdf.title.text_font_size = size_font_title
df4b_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4b_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4b_ecdf.legend.label_text_font_size = size_font_legend
df4b_ecdf.axis.major_label_text_font_size = size_font_tick

# Toolbar_location = None
# bokeh.io.show(df4b_ecdf)


In [13]:
df4c_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     title='Lack of Transfer of NOS2 to CS+ or POS1 to CS-',
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
    y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4c_ecdf.add_layout(Title(
    text='CS+ or POS1 to CS-', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4c_ecdf.add_layout(Title(
    text='Lack of Transfer of NOS2 to', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)

df4c_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4c_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4c,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorG, ColorABC, ColorAB, ColorH],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4c_ecdf
)

#H-
df4c_ecdf.line(
    x=[0,1],
    y=[np.mean(df4cH_values),np.mean(df4cH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='H-',
)

df4c_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4cH_conf_int[1],
    y2=mean_df4cH_conf_int[0],
    color=ColorH,
    legend_label='H-',
    alpha=varea_alpha,
)

#AH
df4c_ecdf.line(
    x=[0,1],
    y=[np.mean(df4cAH_values),np.mean(df4cAH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='AH',
    line_dash='dotted',
)

df4c_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4cAH_conf_int[1],
    y2=mean_df4cAH_conf_int[0],
    color=ColorH,
    legend_label='AH',
    alpha=varea_alpha,
)

#AG
df4c_ecdf.line(
    x=[0,1],
    y=[np.mean(df4cAG_values),np.mean(df4cAG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='AG',
    line_dash='dotted',
)

df4c_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4cAG_conf_int[1],
    y2=mean_df4cAG_conf_int[0],
    color=ColorG,
    legend_label='AG',
    alpha=varea_alpha,
)

#G+
df4c_ecdf.line(
    x=[0,1],
    y=[np.mean(df4cG_values),np.mean(df4cG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='G+',
)

df4c_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4cG_conf_int[1],
    y2=mean_df4cG_conf_int[0],
    color=ColorG,
    legend_label='G+',
    alpha=varea_alpha,
)

df4c_ecdf.title.text_font_size = size_font_title
df4c_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4c_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4c_ecdf.legend.label_text_font_size = size_font_legend
df4c_ecdf.axis.major_label_text_font_size = size_font_tick

# Toolbar_location = None
# bokeh.io.show(df4c_ecdf)


In [14]:
df4d_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     title='Lack of Transfer of NOS2 to POS1/CS Combination or NOS2/POS1 Combination to CS+',
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
    y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4d_ecdf.add_layout(Title(
    text='or NOS2/POS1 to CS+', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4d_ecdf.add_layout(Title(
    text='Lack of Transfer of NOS2 to POS1/CS', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)

df4d_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4d_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4d,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorG, ColorJK, ColorABC, ColorABC, ColorABC],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4d_ecdf
)

#G+
df4d_ecdf.line(
    x=[0,1],
    y=[np.mean(df4dG_values),np.mean(df4dG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='G+',
)

df4d_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4dG_conf_int[1],
    y2=mean_df4dG_conf_int[0],
    color=ColorG,
    legend_label='G+',
    alpha=varea_alpha,
)

#ABG
df4d_ecdf.line(
    x=[0,1],
    y=[np.mean(df4dABG_values),np.mean(df4dABG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='ABG',
    line_dash='dotted',
)

df4d_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4dABG_conf_int[1],
    y2=mean_df4dABG_conf_int[0],
    color=ColorG,
    legend_label='ABG',
    alpha=varea_alpha,
)

#JK+
df4d_ecdf.line(
    x=[0,1],
    y=[np.mean(df4dJK_values),np.mean(df4dJK_values)],
    line_color=ColorJK,
    line_width=line_plot_line_width,
    legend='JK+',
)

df4d_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4dJK_conf_int[1],
    y2=mean_df4dJK_conf_int[0],
    color=ColorJK,
    legend_label='JK+',
    alpha=varea_alpha,
)

#AJK
df4d_ecdf.line(
    x=[0,1],
    y=[np.mean(df4dAJK_values),np.mean(df4dAJK_values)],
    line_color=ColorJK,
    line_width=line_plot_line_width,
    legend='AJK',
    line_dash='dotted',
)

df4d_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4dAJK_conf_int[1],
    y2=mean_df4dAJK_conf_int[0],
    color=ColorJK,
    legend_label='AJK',
    alpha=varea_alpha,
)

#ABC
df4d_ecdf.line(
    x=[0,1],
    y=[np.mean(df4dABC_values),np.mean(df4dABC_values)],
    line_color=ColorABC,
    line_width=line_plot_line_width,
    legend='ABC-',
)

df4d_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4dABC_conf_int[1],
    y2=mean_df4dABC_conf_int[0],
    color=ColorABC,
    legend_label='ABC-',
    alpha=varea_alpha,
)

df4d_ecdf.title.text_font_size = size_font_title
df4d_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4d_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4d_ecdf.legend.label_text_font_size = size_font_legend
df4d_ecdf.axis.major_label_text_font_size = size_font_tick

# Toolbar_location = None
# bokeh.io.show(df4d_ecdf)


In [15]:
df4e_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     title='Successful Transfer: NOS2 to Ambiguous POS1; Lack of Transfer: NOS2 to Unambiguous POS1',
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
    y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4e_ecdf.add_layout(Title(
    text='Lack of Transfer: NOS2 to Unambiguous POS1', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4e_ecdf.add_layout(Title(
    text='Successful Transfer: NOS2 to Ambiguous POS1;', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)

df4e_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4e_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4e,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorJK, ColorABC, ColorJK, ColorTJK, ColorABC],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4e_ecdf
)

#JK2
df4e_ecdf.line(
    x=[0,1],
    y=[np.mean(df4eJK2_values),np.mean(df4eJK2_values)],
    line_color=ColorJK,
    line_width=line_plot_line_width,
    legend='JK+2',
)

df4e_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4eJK2_conf_int[1],
    y2=mean_df4eJK2_conf_int[0],
    color=ColorJK,
    legend_label='JK+2',
    alpha=varea_alpha,
)

#AJK1
df4e_ecdf.line(
    x=[0,1],
    y=[np.mean(df4eAJK1_values),np.mean(df4eAJK1_values)],
    line_color=ColorJK,
    line_width=line_plot_line_width,
    legend='AJK1',
    line_dash='dotted',
)

df4e_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4eAJK1_conf_int[1],
    y2=mean_df4eAJK1_conf_int[0],
    color=ColorJK,
    legend_label='AJK1',
    alpha=varea_alpha,
)

#AJK2
df4e_ecdf.line(
    x=[0,1],
    y=[np.mean(df4eAJK2_values),np.mean(df4eAJK2_values)],
    line_color=ColorTJK,
    line_width=line_plot_line_width,
    legend='AJK2',
    line_dash='dotted',
)

df4e_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4eAJK2_conf_int[1],
    y2=mean_df4eAJK2_conf_int[0],
    color=ColorTJK,
    legend_label='AJK2',
    alpha=varea_alpha,
)

#TJK-2
df4e_ecdf.line(
    x=[0,1],
    y=[np.mean(df4eTJK2_values),np.mean(df4eTJK2_values)],
    line_color=ColorTJK,
    line_width=line_plot_line_width,
    legend='TJK-2',
)

df4e_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4eTJK2_conf_int[1],
    y2=mean_df4eTJK2_conf_int[0],
    color=ColorTJK,
    legend_label='TJK-2',
    alpha=varea_alpha,
)

#ABC-2
df4e_ecdf.line(
    x=[0,1],
    y=[np.mean(df4eABC2_values),np.mean(df4eABC2_values)],
    line_color=ColorABC,
    line_width=line_plot_line_width,
    legend='ABC-2',
)

df4e_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4eABC2_conf_int[1],
    y2=mean_df4eABC2_conf_int[0],
    color=ColorABC,
    legend_label='ABC-2',
    alpha=varea_alpha,
)

df4e_ecdf.title.text_font_size = size_font_title
df4e_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4e_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4e_ecdf.legend.label_text_font_size = size_font_legend
df4e_ecdf.axis.major_label_text_font_size = size_font_tick

# Toolbar_location = None
# bokeh.io.show(df4e_ecdf)


In [16]:
df4f_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     title='Null Transfer of Ambiguous NOS1 to CS+',
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
#     y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4f_ecdf.add_layout(Title(
    text='to CS+', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4f_ecdf.add_layout(Title(
    text='Null Transfer of Unambiguous NOS1', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)

df4f_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4f_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4f,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorG, ColorEF, ColorEF],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4f_ecdf
)

#G+
df4f_ecdf.line(
    x=[0,1],
    y=[np.mean(df4fG_values),np.mean(df4fG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='G+',
)

df4f_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4fG_conf_int[1],
    y2=mean_df4fG_conf_int[0],
    color=ColorG,
    legend_label='G+',
    alpha=varea_alpha,
)

#EG
df4f_ecdf.line(
    x=[0,1],
    y=[np.mean(df4fEG_values),np.mean(df4fEG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='EG',
    line_dash='dotted',
)

df4f_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4fEG_conf_int[1],
    y2=mean_df4fEG_conf_int[0],
    color=ColorG,
    legend_label='EG',
    alpha=varea_alpha,
)

#EF-
df4f_ecdf.line(
    x=[0,1],
    y=[np.mean(df4fEF_values),np.mean(df4fEF_values)],
    line_color=ColorEF,
    line_width=line_plot_line_width,
    legend='EF-',
)

df4f_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4fEF_conf_int[1],
    y2=mean_df4fEF_conf_int[0],
    color=ColorEF,
    legend_label='EF-',
    alpha=varea_alpha,
)

df4f_ecdf.title.text_font_size = size_font_title
df4f_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4f_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4f_ecdf.legend.label_text_font_size = size_font_legend
df4f_ecdf.axis.major_label_text_font_size = size_font_tick

# Toolbar_location = None
# bokeh.io.show(df4f_ecdf)


In [17]:
df4g_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     title='Null Transfer of Ambiguous NOS1 to CS+',
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
#     y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4g_ecdf.add_layout(Title(
    text='to CS+', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4g_ecdf.add_layout(Title(
    text='Null Transfer of Ambiguous NOS1', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)

df4g_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4g_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4g,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorG, ColorMN, ColorMN],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4g_ecdf
)

#G+
df4g_ecdf.line(
    x=[0,1],
    y=[np.mean(df4gG_values),np.mean(df4gG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='G+',
)

df4g_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4gG_conf_int[1],
    y2=mean_df4gG_conf_int[0],
    color=ColorG,
    legend_label='G+',
    alpha=varea_alpha,
)

#MG
df4g_ecdf.line(
    x=[0,1],
    y=[np.mean(df4gMG_values),np.mean(df4gMG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='MG',
    line_dash='dotted',
)

df4g_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4gMG_conf_int[1],
    y2=mean_df4gMG_conf_int[0],
    color=ColorG,
    legend_label='MG',
    alpha=varea_alpha,
)

#MN-
df4g_ecdf.line(
    x=[0,1],
    y=[np.mean(df4gMN_values),np.mean(df4gMN_values)],
    line_color=ColorMN,
    line_width=line_plot_line_width,
    legend='MN-',
)

df4g_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4gMN_conf_int[1],
    y2=mean_df4gMN_conf_int[0],
    color=ColorMN,
    legend_label='MN-',
    alpha=varea_alpha,
)

df4g_ecdf.title.text_font_size = size_font_title
df4g_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4g_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4g_ecdf.legend.label_text_font_size = size_font_legend
df4g_ecdf.axis.major_label_text_font_size = size_font_tick

# Toolbar_location = None
# bokeh.io.show(df4g_ecdf)


In [18]:
df4h_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     title='Lack of Transfer of POS2 to CS- or NOS1 to CS+',
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
#     y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4h_ecdf.add_layout(Title(
    text='or NOS1 to CS+', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4h_ecdf.add_layout(Title(
    text='Lack of Transfer of POS2 to CS-', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)

df4h_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4h_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4h,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorG, ColorDE, ColorDEF, ColorH],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4h_ecdf
)

#H-
df4h_ecdf.line(
    x=[0,1],
    y=[np.mean(df4hH_values),np.mean(df4hH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='H-',
)

df4h_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4hH_conf_int[1],
    y2=mean_df4hH_conf_int[0],
    color=ColorH,
    legend_label='H-',
    alpha=varea_alpha,
)

#DH
df4h_ecdf.line(
    x=[0,1],
    y=[np.mean(df4hDH_values),np.mean(df4hDH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='DH',
    line_dash='dotted',
)

df4h_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4hDH_conf_int[1],
    y2=mean_df4hDH_conf_int[0],
    color=ColorH,
    legend_label='DH',
    alpha=varea_alpha,
)

#DG
df4h_ecdf.line(
    x=[0,1],
    y=[np.mean(df4hDG_values),np.mean(df4hDG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='DG',
    line_dash='dotted',
)

df4h_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4hDG_conf_int[1],
    y2=mean_df4hDG_conf_int[0],
    color=ColorG,
    legend_label='DG',
    alpha=varea_alpha,
)

#G+
df4h_ecdf.line(
    x=[0,1],
    y=[np.mean(df4hG_values),np.mean(df4hG_values)],
    line_color=ColorG,
    line_width=line_plot_line_width,
    legend='G+',
)

df4h_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4hG_conf_int[1],
    y2=mean_df4hG_conf_int[0],
    color=ColorG,
    legend_label='G+',
    alpha=varea_alpha,
)

df4h_ecdf.title.text_font_size = size_font_title
df4h_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4h_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4h_ecdf.legend.label_text_font_size = size_font_legend
df4h_ecdf.axis.major_label_text_font_size = size_font_tick

# Toolbar_location = None
# bokeh.io.show(df4h_ecdf)


In [19]:
df4i_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     title='Lack of Transfer of NOS2 to POS1/CS Combination or NOS2/POS1 Combination to CS+',
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
#     y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4i_ecdf.add_layout(Title(
    text='or POS2/NOS1 to CS-', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4i_ecdf.add_layout(Title(
    text='Lack of Transfer of POS2 to NOS1/CS', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)

df4i_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4i_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4i,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorDEF, ColorDEF, ColorDEF, ColorMN, ColorH],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4i_ecdf
)

#H-
df4i_ecdf.line(
    x=[0,1],
    y=[np.mean(df4iH_values),np.mean(df4iH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='H-',
)

df4i_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4iH_conf_int[1],
    y2=mean_df4iH_conf_int[0],
    color=ColorH,
    legend_label='H-',
    alpha=varea_alpha,
)

#DEH
df4i_ecdf.line(
    x=[0,1],
    y=[np.mean(df4iDEH_values),np.mean(df4iDEH_values)],
    line_color=ColorH,
    line_width=line_plot_line_width,
    legend='DEH',
    line_dash='dotted',
)

df4i_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4iDEH_conf_int[1],
    y2=mean_df4iDEH_conf_int[0],
    color=ColorH,
    legend_label='DEH',
    alpha=varea_alpha,
)

#MN-
df4i_ecdf.line(
    x=[0,1],
    y=[np.mean(df4iMN_values),np.mean(df4iMN_values)],
    line_color=ColorMN,
    line_width=line_plot_line_width,
    legend='MN-',
)

df4i_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4iMN_conf_int[1],
    y2=mean_df4iMN_conf_int[0],
    color=ColorMN,
    legend_label='MN-',
    alpha=varea_alpha,
)

#DMN
df4i_ecdf.line(
    x=[0,1],
    y=[np.mean(df4iDMN_values),np.mean(df4iDMN_values)],
    line_color=ColorMN,
    line_width=line_plot_line_width,
    legend='DMN',
    line_dash='dotted',
)

df4i_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4iDMN_conf_int[1],
    y2=mean_df4iDMN_conf_int[0],
    color=ColorMN,
    legend_label='DMN',
    alpha=varea_alpha,
)

#DEF+
df4i_ecdf.line(
    x=[0,1],
    y=[np.mean(df4iDEF_values),np.mean(df4iDEF_values)],
    line_color=ColorDEF,
    line_width=line_plot_line_width,
    legend='DEF+',
)

df4i_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4iDEF_conf_int[1],
    y2=mean_df4iDEF_conf_int[0],
    color=ColorDEF,
    legend_label='DEF+',
    alpha=varea_alpha,
)

df4i_ecdf.title.text_font_size = size_font_title
df4i_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4i_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4i_ecdf.legend.label_text_font_size = size_font_legend
df4i_ecdf.axis.major_label_text_font_size = size_font_tick

# Toolbar_location = None
# bokeh.io.show(df4i_ecdf)


In [20]:
df4j_ecdf = bokeh.plotting.figure(
    frame_width=450,
    frame_height=300,
#     title='Successful Transfer: POS2 to Ambiguous NOS1; Lack of Transfer: POS2 to Unambiguous NOS1',
#     x_axis_label='Fraction of Observations Less Than Or Equal to US Expectancy Value',
#     y_axis_label='US Expectancy (1-5)',
    x_range=(-.4,1.05),
    y_range=(.9,5.1),
)

#Title Bottom Line
df4j_ecdf.add_layout(Title(
    text='Lack of Transfer: POS2 to Unambiguous NOS1', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)
#Title Top Line
df4j_ecdf.add_layout(Title(
    text='Successful Transfer: POS2 to Ambiguous NOS1;', 
    text_font_style="normal", 
    text_font_size=size_font_title,
    align='center',), 
    'above'
)

df4j_ecdf.yaxis.axis_label_text_font_style = 'normal'
df4j_ecdf.xaxis.axis_label_text_font_style = 'normal'

iqplot.ecdf(
    data=df4j,
    q='US Exp',
    cats='Stimulus',
    palette=[ColorDEF, ColorUMN, ColorMN, ColorDEF, ColorMN],
    marker='circle',
    marker_kwargs=(dict(
        alpha=1,
        line_color='BorderColor',
        size=4.25,
        line_width=1.25,
    )),
    q_axis='y',
    conf_int=False,
    show_legend=True,  
    tooltips=[
        ('US Exp', '@{US Exp}'),
        ('Stimulus', '@{Stimulus}'),
        
    ],
    p=df4j_ecdf
)

#MN-2
df4j_ecdf.line(
    x=[0,1],
    y=[np.mean(df4jMN2_values),np.mean(df4jMN2_values)],
    line_color=ColorMN,
    line_width=line_plot_line_width,
    legend='MN-2',
)

df4j_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4jMN2_conf_int[1],
    y2=mean_df4jMN2_conf_int[0],
    color=ColorMN,
    legend_label='MN-2',
    alpha=varea_alpha,
)

#DMN1
df4j_ecdf.line(
    x=[0,1],
    y=[np.mean(df4jDMN1_values),np.mean(df4jDMN1_values)],
    line_color=ColorMN,
    line_width=line_plot_line_width,
    legend='DMN1',
    line_dash='dotted',
)

df4j_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4jDMN1_conf_int[1],
    y2=mean_df4jDMN1_conf_int[0],
    color=ColorMN,
    legend_label='DMN1',
    alpha=varea_alpha,
)

#DMN2
df4j_ecdf.line(
    x=[0,1],
    y=[np.mean(df4jDMN2_values),np.mean(df4jDMN2_values)],
    line_color=ColorUMN,
    line_width=line_plot_line_width,
    legend='DMN2',
    line_dash='dotted',
)

df4j_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4jDMN2_conf_int[1],
    y2=mean_df4jDMN2_conf_int[0],
    color=ColorUMN,
    legend_label='DMN2',
    alpha=varea_alpha,
)

#UMN-2
df4j_ecdf.line(
    x=[0,1],
    y=[np.mean(df4jUMN2_values),np.mean(df4jUMN2_values)],
    line_color=ColorUMN,
    line_width=line_plot_line_width,
    legend='UMN+2',
)

df4j_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4jUMN2_conf_int[1],
    y2=mean_df4jUMN2_conf_int[0],
    color=ColorUMN,
    legend_label='UMN+2',
    alpha=varea_alpha,
)

#DEF+2
df4j_ecdf.line(
    x=[0,1],
    y=[np.mean(df4jDEF2_values),np.mean(df4jDEF2_values)],
    line_color=ColorDEF,
    line_width=line_plot_line_width,
    legend='DEF+2',
)

df4j_ecdf.varea(
    x=np.linspace(0,1),
    y1=mean_df4jDEF2_conf_int[1],
    y2=mean_df4jDEF2_conf_int[0],
    color=ColorDEF,
    legend_label='DEF+2',
    alpha=varea_alpha,
)

df4j_ecdf.title.text_font_size = size_font_title
df4j_ecdf.xaxis.axis_label_text_font_size = size_font_label
df4j_ecdf.yaxis.axis_label_text_font_size = size_font_label
df4j_ecdf.legend.label_text_font_size = size_font_legend
df4j_ecdf.axis.major_label_text_font_size = size_font_tick
df4e_ecdf.xaxis.axis_label = 'Fraction of Observations ≤ US Expectancy Value'
df4j_ecdf.xaxis.axis_label = 'Fraction of Observations ≤ US Expectancy Value'

# bokeh.io.show(df4j_ecdf)


In [24]:
plots = [df4a_ecdf, df4b_ecdf, df4c_ecdf, df4d_ecdf, df4e_ecdf, df4f_ecdf, df4g_ecdf, df4h_ecdf, df4i_ecdf, df4j_ecdf]

grid = gridplot(
    [
    [df4a_ecdf, df4f_ecdf],
    [df4b_ecdf, df4g_ecdf],
    [df4c_ecdf, df4h_ecdf],
    [df4d_ecdf, df4i_ecdf],
    [df4e_ecdf, df4j_ecdf],
    ],
#     toolbar_location=None
)

bokeh.io.show(grid)

bokeh.io.save(
    grid,
    filename=os.path.join('../output/',r'2nd OS - Figure 4.html'),
    title='Figure 4'
)


'C:\\Users\\tzboz\\Desktop\\Postdoc\\Research\\Papers\\Second-Order Occasion Setting\\2nd OS Repository\\figures\\output\\2nd OS - Figure 4.html'

In [26]:
%load_ext watermark
%watermark -v -p pytest,jupyterlab,bokeh,iqplot,bebi103,numpy,pandas

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
CPython 3.8.5
IPython 7.18.1

pytest 6.0.2
jupyterlab 2.2.6
bokeh 2.2.1
iqplot 0.1.6
bebi103 0.1.1
numpy 1.19.1
pandas 1.1.1
